In [4]:
import nest_asyncio
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import whisper
from transformers import pipeline
import os
import uvicorn
import warnings
import logging

In [6]:
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")
warnings.filterwarnings("ignore", message="Using a GPU for inference is not supported; using CPU instead")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

nest_asyncio.apply() #to run Fast API on jupyter
app = FastAPI()


In [7]:
#---------------------Loading Models -------------------

try:
    whisper_model = whisper.load_model("large-v3")  
    logger.info("Whisper model loaded successfully.")
    
except Exception as e:
    logging.error(f"Error loading Whisper model: {str(e)}")
    raise RuntimeError(f"Error loading Whisper model: {str(e)}")


try:
    summary_pipeline = pipeline("summarization")
    logger.info("Summarization pipeline loaded successfully.")
    
except Exception as e:
    logging.error(f"Error loading summarization pipeline: {str(e)}")
    raise RuntimeError(f"Error loading summarization pipeline: {str(e)}")


INFO:__main__:Whisper model loaded successfully.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\admin\anaconda3\anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:__main__:Summarization pipeline loaded successfully.


In [8]:
UPLOAD_DIR = "./uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

#---------------------saving uploaded file  ----------------

def save_upload_file(upload_file: UploadFile, destination: str) -> str:
    try:
        with open(destination, "wb") as buffer:
            buffer.write(upload_file.file.read())
            
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"File save error: {str(e)}")
        
    return destination

def remove_file(file_path: str):
    if os.path.exists(file_path):
        os.remove(file_path)


In [9]:
#-------------- Transcribes uploaded audio using Whisper model- ---------
@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    file_location = os.path.join(UPLOAD_DIR, file.filename)
    try:
        await save_upload_file(file, file_location)
        loop = asyncio.get_event_loop()
        transcription_result = await loop.run_in_executor(None, whisper_model.transcribe, file_location)
        transcription = transcription_result['text']
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Transcription error: {str(e)}")
        
    finally:
        remove_file(file_location)
        
    return {"transcription": transcription}


#------------ Summarize the transcription of audio file--------------
@app.post("/summarize/")
async def summarize_text(transcription: str):
    try:
        summary = summarizer(transcription, max_length=100, min_length=25, do_sample=False)[0]['summary_text']
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Summarization error: {str(e)}")
    return {"summary": summary}


#------------------Extracting Time stamps from the audio file ----------
@app.post("/extract-timestamps/")
async def extract_timestamps(file: UploadFile = File(...)):
    file_location = os.path.join(UPLOAD_DIR, file.filename)
    try:
        save_upload_file(file, file_location)
        transcription_result = whisper_model.transcribe(file_location)
        segments = transcription_result['segments']
        timestamps = [{"start": seg['start'], "end": seg['end'], "text": seg['text']} for seg in segments]
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Timestamp extraction error: {str(e)}")
        
    finally:
        remove_file(file_location)
        
    return {"timestamps": timestamps}

@app.post("/process-audio/")
async def process_audio(file: UploadFile = File(...)):
    file_location = os.path.join(UPLOAD_DIR, file.filename)
    try:
        save_upload_file(file, file_location)
        transcription_result = whisper_model.transcribe(file_location)
        transcription = transcription_result['text']
        segments = transcription_result['segments']
        summary = summarizer(transcription, max_length=100, min_length=25, do_sample=False)[0]['summary_text']
        timestamps = [{"start": seg['start'], "end": seg['end'], "text": seg['text']} for seg in segments]
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Processing error: {str(e)}")
    finally:
        remove_file(file_location)
    return {"transcription": transcription, "summary": summary, "timestamps": timestamps}



In [ ]:
#------------ Running the Fast API application on port 8000-------------
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [15508]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
